# Import Everthing We Need


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Import data files
movie_budgets_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz') #Michael's from Rotten Tomatos 
movie_studios = pd.read_csv('zippedData/bom.movie_gross.csv.gz')     #Teigen's from Box Office Mojo 
title_basics = pd.read_csv('zippedData/imdb.title.basics.csv.gz')    #Louis' from basics IMDB
title_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')  #Louis' ratings from IMDB
tmdb = pd.read_csv('zippedData/tmdb.movies.csv.gz')                  #Ely's from TMDB

In [ ]:
#Ely's innitial cleaning TMDB 
genres={'28':"Action",
'12':"Adventure",
'16':"Animation",
'35':"Comedy",
'80':"Crime",
'99':"Documentary",
'18':"Drama",
'10751':"Family",
'14':"Fantasy",
'36':"History",
'27':"Horror",
'10402':"Music",
'9648':"Mystery",
'10749':"Romance",
'878':"Science Fiction",
'10770':"TV Movie",
'53':"Thriller",
'10752':"War",
'37':"Western"}

tmdb = tmdb.replace(pd.Series(genres).astype(str), regex=True)
tmdb['genre_ids']=tmdb['genre_ids'].str.strip('[]').str.split(', ')


In [ ]:
#IMDB merge
title_master = title_basics.merge(title_ratings, how='inner')

In [ ]:
louis_and_ely = tmdb.merge(title_master, how = 'inner')
louis_and_ely

In [ ]:
louis_and_ely_and_teigen = louis_and_ely.merge(movie_studios, how = 'inner')


In [ ]:
let = louis_and_ely_and_teigen.drop(['domestic_gross', 'foreign_gross'], axis = 1)
let

In [ ]:
let = louis_and_ely_and_teigen.drop(['domestic_gross', 'foreign_gross'], axis = 1)
let

In [ ]:
#add director info
title_principal=pd.read_csv('zippedData/imdb.title.principals.csv.gz')
title_principal

In [ ]:
title_principal_directors=title_principal[title_principal['category'] == 'director']
title_principal_directors

In [ ]:
name_basics=pd.read_csv('zippedData/imdb.name.basics.csv.gz')
name_basics

In [ ]:
nconst_to_name=dict(zip(list(name_basics['nconst']),list(name_basics['primary_name'])))

In [ ]:
title_principal_directors['director_name']=title_principal_directors['nconst'].map(nconst_to_name)

In [ ]:
title_principal_directors

In [ ]:
let=let.merge(title_principal_directors, on = 'tconst')
let

In [ ]:
movie_budgets_df = movie_budgets_df.rename(columns={'movie':'title'})

In [ ]:
master_unclean = let.merge(movie_budgets_df, on = 'title', how = 'inner')
master_unclean

In [ ]:
master_unclean.info()

In [ ]:
master_unclean['vote_count'].describe()

In [ ]:
run_time = master_unclean.loc[(master_unclean['runtime_minutes'] > 30) & (master_unclean['runtime_minutes'] < 180)]
run_time

In [ ]:
less_columns = master_unclean.drop(['Unnamed: 0', 'id_x', 'original_title', 'tconst', 'start_year', 'id_y', 'genres', 'release_date_x', 'primary_title'], axis = 1)

In [ ]:
less_columns

In [ ]:
less_columns=less_columns.loc[less_columns['year'] >= 2016]

In [ ]:
less_columns=less_columns.drop(['year'], axis=1)

In [ ]:
less_columns

In [ ]:
#clean up budget and gross numbers, add the return on investment column
less_columns['production_budget'] = less_columns['production_budget'].str.replace(',', '')
less_columns['production_budget'] = less_columns['production_budget'].str.replace('$', '')
less_columns['production_budget'] = less_columns['production_budget'].astype(int)
less_columns['domestic_gross'] = less_columns['domestic_gross'].str.replace(',', '').str.replace('$', '')
less_columns['domestic_gross'] = less_columns['domestic_gross'].astype(int)
less_columns['worldwide_gross'] = less_columns['worldwide_gross'].str.replace(',', '').str.replace('$', '')
less_columns['worldwide_gross'] = less_columns['worldwide_gross'].astype(int)
less_columns['total_gross'] = less_columns['domestic_gross'] + less_columns['worldwide_gross']
less_columns['ROI'] = (less_columns['total_gross'] / less_columns['production_budget']) * 100

In [ ]:
less_columns.groupby('studio').mean()

In [ ]:
#drop duplicates
less_columns=less_columns.drop_duplicates(['title','release_date_y'])


In [ ]:
exploded = less_columns.explode('genre_ids', ignore_index=False)

In [ ]:
exploded.reset_index()

In [ ]:
exploded['release_date_y'].values[0][0:3]

In [ ]:
exploded['month'] = exploded['release_date_y'].str[0:3]

In [ ]:
exploded

In [ ]:
month = exploded['month']

In [ ]:
month.value_counts().index

In [ ]:
exploded['genre_ids'].value_counts

In [ ]:
exploded

In [ ]:
month_genre_roi_title = exploded.drop(['original_language', 'popularity', 'vote_average', 'vote_count',\
                                      'runtime_minutes', 'averagerating', 'numvotes', 'studio', \
                                      'production_budget', 'domestic_gross', 'worldwide_gross',\
                                      'total_gross'], axis=1)

In [ ]:
month_genre_roi_title

In [ ]:
#dictionary of months and number
month_to_number={'Jan': 1,'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

In [ ]:
#map month to number in dataframe month_genre_roi_title
month_genre_roi_title['month']=month_genre_roi_title['month'].map(month_to_number)


In [ ]:
month_genre_roi_title

In [ ]:
genre_roi=month_genre_roi_title.groupby(['genre_ids']).mean().sort_values('ROI', ascending=False)
genre_roi

In [ ]:

#take top 5 genres
top_5_genres=list(genre_roi.index[0:5])
top_5_genres

In [ ]:
month_genre_roi_title[month_genre_roi_title['genre_ids'].isin(top_5_genres)]


In [ ]:
month_genre_roi_title=month_genre_roi_title[month_genre_roi_title['genre_ids'].isin(top_5_genres)]

In [ ]:
# set plot style: grey grid in the background:
sns.set(style="darkgrid")

# Set the figure size
plt.figure(figsize=(10, 10))

# grouped barplot
sns.lineplot(x="month", y="ROI", hue="genre_ids", data=month_genre_roi_title , ci=None);

In [ ]:
month_genre_roi_title.groupby(['month','genre_ids']).mean()

In [ ]:
month_genre_roi_title.groupby(['genre_ids','month']).sum()

In [ ]:
month_genre_roi_title['genre_ids'].value_counts()

In [ ]:
groupedby=month_genre_roi_title.groupby(['genre_ids','month']).sum()

In [ ]:
groupedby

#trying to figure out how to get director/writer data

In [ ]:
movie_info_dataframe = pd.read_table('zippedData/rt.movie_info.tsv.gz')
movie_info_dataframe.head()

In [ ]:
rotten_tomato = pd.read_csv('zippedData/rt.reviews.tsv.gz', sep='\t', encoding='latin-1')
rotten_tomato

In [ ]:
title_principal=pd.read_csv('zippedData/imdb.title.principals.csv.gz')
title_principal

In [ ]:
title_principal_directors=title_principal[title_principal['category'] == 'director']
title_principal_directors

In [ ]:
name_basics=pd.read_csv('zippedData/imdb.name.basics.csv.gz')
name_basics

In [ ]:
list(name_basics['nconst'])

In [ ]:
list(name_basics['primary_name'])

In [ ]:
nconst_to_name=dict(zip(list(name_basics['nconst']),list(name_basics['primary_name'])))

In [ ]:
title_principal_directors['director_name']=title_principal_directors['nconst']


In [ ]:
title_principal_directors['director_name']=title_principal_directors['nconst'].map(nconst_to_name)

In [ ]:
title_principal_directors

In [ ]:
title_crew=pd.read_csv('zippedData/imdb.title.crew.csv.gz')
title_crew

In [ ]:
title_basics

In [ ]:
title_basics_

In [ ]:
title_ratings

In [ ]:
name_basics=pd.read_csv('zippedData/imdb.title.principals.csv.gz')
name_basics

In [ ]:
groupedby['ROI']